# Conceptos matemáticos básicos para la interferometría

En este sección, presentaremos los elementos matemáticos esenciales que se encuentran típicamente en el radar de apertura sintética (SAR) y la interferometría. Asumiremos que el lector tiene una comprensión del álgebra. Esta sección puede servir como indicador para que los estudiantes evalúen su capacidad para aprovechar al máximo este curso.

Para esquematizar de una mejor manera todos los conceptos, hemos separado los contenidos de esta sección en los siguientes temas:

1. [Trigonometría](#section-1)<br>
2. [Números complejos](#section-2)<br>
3. [Vectores y algebra vectorial](#section-3)<br>
4. [Algebra lineal o matricial](#section-4)<br>
5. [Calculo elemental](#section-5)<br>
6. [Transformada de Fourier](#section-6)<br>
7. [Decibeles](#section-7)<br>


## Preliminares

Antes de comenzar con la sección daremos importaremos algunas librerías y definiremos algunas funciones que nos serán necesarias para el desarrollo de la sección

In [1]:
# Algunas librerías necesarias y definiciones de parámetros

import warnings
warnings.filterwarnings('ignore')

bShowInline = True  # Set = False para la generación de documentos
%matplotlib inline
import matplotlib.pyplot as plt

params = {'legend.fontsize': 'x-large',
          'figure.figsize': (15, 5),
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'x-large'}
plt.rcParams.update(params)

def makeplot( plt, figlabel, figcaption):
    figname = figlabel+'.png'
    plt.savefig(figname)
    if bShowInline:
        plt.show()
    else:
        plt.close()
    strLatex="""
    \\begin{figure}[b]
    \centering
        \includegraphics[totalheight=10.0cm]{%s}
        \caption{%s}
        \label{fig:%s}
    \end{figure}"""%(figname, figcaption, figlabel) 
    return display(Latex(strLatex)) 

In [2]:
from ipywidgets import widgets, Layout, Box, GridspecLayout

def create_multipleChoice_widget(description, options, correct_answer, hint):
    if correct_answer not in options:
        options.append(correct_answer)
    correct_answer_index = options.index(correct_answer)
    radio_options = [(words, i) for i, words in enumerate(options)]
    alternativ = widgets.RadioButtons(
        options = radio_options,
        description = '',
        disabled = False,
        indent = False,
        align = 'center',
    )
    description_out = widgets.Output(layout=Layout(width='auto'))
    with description_out:
        print(description)
    feedback_out = widgets.Output()
    def check_selection(b):
        a = int(alternativ.value)
        if a==correct_answer_index:
            s = '\x1b[6;30;42m' + "correct" + '\x1b[0m' +"\n"
        else:
            s = '\x1b[5;30;41m' + "try again" + '\x1b[0m' +"\n"
        with feedback_out:
            feedback_out.clear_output()
            print(s)
        return
    check = widgets.Button(description="check")
    check.on_click(check_selection)
    hint_out = widgets.Output()
    def hint_selection(b):
        with hint_out:
            print(hint)       
        with feedback_out:
            feedback_out.clear_output()
            print(hint)
    hintbutton = widgets.Button(description="hint")
    hintbutton.on_click(hint_selection)
    return widgets.VBox([description_out, 
                         alternativ, 
                         widgets.HBox([hintbutton, check]), feedback_out], 
                        layout=Layout(display='flex',
                                     flex_flow='column',
                                     align_items='stretch',
                                     width='auto')) 
test = create_multipleChoice_widget('1. Let a, b, c be real numbers. If a<b and b<c, then:',
                                   ['a>c','a<c','b<c','c<a'],
                                   'a<c',
                                   "[hint]:")
trig = create_multipleChoice_widget('',
    ['A','B','C','D'],
    'B',"[hint]: Given in question")
comp1 = create_multipleChoice_widget('',
    ['A','B','C','D'],
    'C',"[hint]:")
comp2 = create_multipleChoice_widget('',
    ['A','B','C','D'],
    'B',"[hint]:")
vec1 = create_multipleChoice_widget('',
    ['A','B','C','D'],
    'B',"[hint]:")
vec2 = create_multipleChoice_widget('',
    ['A','B','C','D'],
    'A',"[hint]:")
calc = create_multipleChoice_widget('',
    ['A','B','C','D'],
    'A',"[hint]:")
dB = create_multipleChoice_widget('',
    ['A','B','C','D'],
    'D',"[hint]:")

<a id="section-1"></a>
## Trigonometría

Un aporte importante de la trigonometría es el estudio de triángulos. En las imágenes de radar y la interferometría nos encontramos con muchos triángulos. Por lo que haremos un recordatorio de algunos conceptos básicos.

Definamos un triángulo rectángulo (uno con un ángulo de $90^\circ$ entre dos de los lados) donde el lado horizontal tiene longitud $x$ y el lado vertical tiene longitud $y$. Podemos decir que la longitud tercer lado (llamado hipotenusa) está dada por $\sqrt{x^2 + y^2}$, como se ilustra en la figura.

<div>
<p style = 'text-align:center;'>
<img src="Figs/triangle.png" width="300"/> 
</p>
</div>



Con el ángulo $\theta$ entre la horizontal y la hipotenusa especificado, podemos definir las relaciones trigonométricas entre los lados y el ángulo anteriormente definido. Estas relaciones son las siguientes:

\begin{equation}
\cos \theta = \frac{x}{\sqrt{x^2 + y^2}}
\end{equation}

\begin{equation}
\sin \theta = \frac{y}{\sqrt{x^2 + y^2}}
\end{equation}

\begin{equation}
\tan \theta = \frac{y}{x}
\end{equation}

\begin{equation}
\sec \theta = \frac{1}{\cos \theta}
\end{equation}

\begin{equation}
\csc \theta = \frac{1}{\sin \theta}
\end{equation}

\begin{equation}
\cot \theta = \frac{1}{\tan \theta}
\end{equation}

### El circulo unitario

Otra forma de definir las funciones trigonométricas es a través del círculo unitario. Para un círculo de radio $r$, la circunferencia viene dada por

\begin{equation}
C=2\pi r
\end{equation}

Llamaremos "círculo unitario" al cirvulo cuyo radio es $r=1$, como se ilustra en la imagen que se presenta a continuación. La circunferencia es $2\pi$ y cualquier punto $P$ en el círculo está descrito por las coordenadas cartesianas $(x,y)$ $(\cos\theta,\sin\theta)$ donde $\theta$ es el ángulo formado por la horizontal y la línea desde el origen hasta $P$. El ángulo $\theta$ se mide desde la horizontal. Las coordenadas $x=\cos\theta, y=\sin\theta$ oscilan entre $+1$ y $-1$ a medida que aumenta $\theta$.

<div>
<p style = 'text-align:center;'>
<img src="Figs/unitcircle.jpg" width="400"/> 
</p>

### Identidades trigonométricas básicas

Hay una gran cantidad de relaciones o identidades útiles que se usan explícitamente en este curso o que pueden ser útiles para resolver otros problemas relacionados. Acá dejamos una lista de las identidades más clásicas

\begin{equation}
\sin^2\theta + \cos^2\theta = 1  \qquad , \qquad \sin(-\theta) = -\sin(\theta) \qquad , \qquad\cos(-\theta) = \cos(\theta)
\end{equation}

\begin{equation}
 \sin(2\alpha) = 2\sin(\alpha)\cos(\alpha) \qquad , \qquad \cos(2\alpha) = \cos^2\alpha -\sin^2\alpha
\end{equation}

\begin{equation}
 \sin\left(\frac{\alpha}{2}\right) = \pm \sqrt{\frac{1-\cos\alpha}{2}} \qquad , \qquad   \cos\left(\frac{\alpha}{2}\right) = \pm \sqrt{\frac{1+\cos\alpha}{2}}
\end{equation}

\begin{equation}
1+\tan^2\alpha = \sec^2\alpha \qquad , \qquad 1+\cot^2\alpha = \csc^2\alpha 
\end{equation}

\begin{equation}
\sin(\alpha\pm\beta) = \sin\alpha\cos\beta \pm \cos\alpha \sin\beta 
\end{equation}

\begin{equation}
\cos(\alpha\pm\beta) = \cos\alpha\cos\beta \mp \sin\alpha \sin\beta 
\end{equation}

\begin{equation}
\tan(\alpha\pm\beta) = \frac{\tan\alpha \pm \tan\beta}{1\mp\tan\alpha\tan\beta}
\end{equation}


## Sumas trigonométricas y relaciones de productos

A veces es conveniente convertir sumas de cantidades trigonométricas en productos y viceversa. Aquí hay algunas fórmulas convenientes.

\begin{equation}
\sin\theta\sin\phi = \frac{\cos(\theta-\phi)-\cos(\theta+\phi)}{2}
\end{equation}

\begin{equation}
\sin\theta\cos\phi = \frac{\sin(\theta+\phi)-\sin(\theta-\phi)}{2}
\end{equation}

\begin{equation}
\cos\theta\cos\phi = \frac{\cos(\theta-\phi)+\cos(\theta+\phi)}{2}
\end{equation}

\begin{equation}
\sin\theta\pm \sin\phi = 2\sin\left(\frac{\theta\pm\phi}{2}\right)\cos\left(\frac{\theta\mp\phi}{2}\right)
\end{equation}

\begin{equation}
\cos\theta + \cos\phi = 2\cos\left(\frac{\theta + \phi}{2}\right)\cos\left(\frac{\theta-\phi}{2}\right)
\end{equation}

\begin{equation}
\cos\theta - \cos\phi = - 2\sin\left(\frac{\theta + \phi}{2}\right)\sin\left(\frac{\theta-\phi}{2}\right)
\end{equation}

## Leyes del seno y el coseno

Las llamadas leyes del seno y del coseno en trigonometría se utilizan mucho en la interferometría. Permiten la solución fácil para los ángulos o lados de un triángulo de forma arbitraria, como se muestra aquí.

<br> <img src="Figs/gentriangle.png" width="400" align="center" /> <br> 

### Ley de los senos

\begin{equation}
\frac{\sin\alpha}{a} = \frac{\sin\beta}{b} = \frac{\sin\gamma}{c}
\end{equation}

### Ley de los cosenos

\begin{equation}
c^2 = a^2 + b^2 - 2 a b \cos \gamma
\end{equation}

\begin{equation}
a^2 = b^2 + c^2 - 2 b c \cos \alpha
\end{equation}

\begin{equation}
b^2 = a^2 + c^2 - 2 a c \cos \beta
\end{equation}

Tenga en cuenta que si uno de los ángulos es $90^\circ$, digamos $\gamma$ en este caso, entonces la Ley de los cosenos se reduce al teorema de Pitágoras:

\begin{equation}
c^2 = a^2 + b^2
\end{equation}

<a id="section-2"></a>
## Números Complejos 

### ¿Qué es un número complejo y realmente los necesitamos?

La respuesta es sí. Los números complejos surgieron en el siglo XVI al intentar resolver ecuaciones polinómicas como

\begin{equation}
x^2+1 = 0
\end{equation}

Se encontró que al agregar $i=\sqrt{-1}$ era posible determinar todas las soluciones de $n$ para cualquier ecuación polinomial de orden $n$ ( $n=2$ en la ecuación anterior).

En física e ingeniería, los números complejos surgen de forma natural en una miríada de formas, incluida la descripción de fenómenos ondulatorios. Los píxeles en las imágenes de radar se representan fundamentalmente como números complejos, cada uno con una amplitud que representa la reflectividad del píxel y una fase que codifica la geometría de la observación, así como otros efectos de superficie. Como tal, la comprensión de los números complejos es esencial para comprender completamente las representaciones subyacentes y el lenguaje de la interferometría.

Afortunadamente, solo se necesitan algunos datos básicos e identidades sobre números complejos.

Es importante observar que

\begin{equation}
i^2 = -1  \qquad \Rightarrow \qquad \frac{1}{i} = - i
\end{equation}


### Aritmética compleja

Todos los números complejos se pueden escribir en la forma:

\begin{equation}
z = \underbrace{\quad x \quad}_\text{parte real} + \quad i \underbrace{\quad  y \quad}_\text{parte imaginaria}
\end{equation}

Para sumar o restar números complejos, simplemente suma o resta las partes real e imaginaria. Para $z_1 = x_1 + i y_1$ y $z_2 = x_2 + i y_2$,

\begin{equation}
z_1 \pm z_2 = (x_1 \pm x_2) + i (y_1 \pm y_2)
\end{equation}





### Vista geométrica de números complejos

#### El plano complejo

Los números complejos se pueden ver como puntos en un plano, donde los ejes de coordenadas del plano son los ejes real e imaginario. La suma y la resta se pueden visualizar como una suma de vectores, como se muestra en la siguiente figura.

<br> <img src="Figs/complexplane.png" width="400" align="center" /> <br> 



#### Módulo (o Magnitud) y Conjugados de un Número Complejo

La magnitud o módulo de un número complejo $z$ es simplemente su longitud en el plano complejo y normalmente se expresa como $\vert z \vert$.

La magnitud de $z$ se puede calcular en términos de su complejo conjugado, expresado como $z^*$ o $\bar{z}$. Esto es simplemente su reflejo sobre el eje real.

<br> <img src="Figs/conjugate.png" width="400" align="center" /> <br> 

De la figura, es fácil ver que la parte real $\Re (z)$ y la parte imaginaria $\Im(z)$ de $z$ son
\begin{equation}
\Re(z) = \frac{z+z^*}{2} \qquad \Im(z) = \frac{z-z^*}{2i}
\end{equation}

La magnitud de $z$ geométricamente la hipotenusa del triángulo formado entre $x$, $y$ y $\vert z \vert$ en el plano complejo:
\begin{equation}
\vert z \vert = \sqrt{x^2+y^2}
\end{equation}

Más formalmente, la magnitud de un vector complejo se define como
\begin{equation}
\vert z \vert  =  \sqrt{zz^*} = \sqrt{x^2+y^2}
\end{equation}

### Multiplicación y División de Números Complejos

Para multiplicar y dividir números complejos, los tratamos como binomios. La multiplicación es sencilla:

\begin{equation}
z_1 z_2  =  x_1 x_2 - y_1 y_2 + i(x_1 y_2 + x_2 y_1)
\end{equation}

La división es un poco más complicada. Aquí cambiamos la división en multiplicación al multiplicar el numerador y el denominador por el complejo conjugado del denominador
\begin{equation}
\frac{z_1}{z_2} =  \frac{x_1 x_2 + y_1 y_2}{x^2+y^2} + i \frac{-x_1 y_2 + x_2 y_1}{x^2+y^2}
\end{equation}


### Representación en formato polar de números complejos

Tal como vimos con los triángulos y las funciones trigonométricas, las componentes real e imaginaria de un número complejo se pueden representar en un formato polar (ahí examinamos el círculo unitario donde la hipotenusa siempre tuvo longitud 1. Aquí el número complejo tiene longitud $r = |z|$.) Haciendo referencia a la figura,
\begin{equation}
z_1 = x_1 + i y_1 = r_1 \cos\theta_1 + i r_1 \sin\theta_1
\end{equation}

\begin{equation}
z_2 = x_2 + i y_2 = r_2 \cos\theta_2 + i r_2 \sin\theta_2
\end{equation}

<br> <img src="Figs/polar.png" width="300" align="center" /> <br> 

Esta representación proporciona una vista geométrica de la multiplicación y división que simplifica el esfuerzo.

\begin{equation}
z_1 z_2  =  r_1 r_2 \cos(\theta_1+\theta_2) + i r_1 r_2 \sin(\theta_1+\theta_2)
\end{equation}

Entonces el producto involucra el producto de las magnitudes y las sumas de los ángulos. Una expansión similar para la división muestra que la razón involucra la razón de las magnitudes y las diferencias de los ángulos:
\begin{equation}
\frac{z_1}{z_2} = \frac{r_1}{r_2} \cos(\theta_1-\theta_2) + i \frac{r_1}{r_2} \sin(\theta_1-\theta_2) 
\end{equation}


### Identidades de Euler

Basándose en las fórmulas de la forma polar para la multiplicación y la división, Euler desarrolló la siguiente identidad notable:

\begin{equation}
e^{i\theta} = \cos\theta + i \sin\theta
\end{equation}

Entonces todo número complejo se puede escribir en la forma

\begin{equation}
z = r\cos\theta + i r\sin\theta = r e^{i\theta}
\end{equation}

La multiplicación y la división son fáciles en este formato.

\begin{equation}
z_1 z_2 = r_1 e^{i\theta_1} r_2 e^{i\theta_2} = r_1 r_2 e^{i(\theta_1+\theta_2)}
\end{equation}

\begin{equation}
\frac{z_1}{z_2} = \frac{r_1 e^{i\theta_1}}{r_2 e^{i\theta_2}} = \frac{r_1}{r_2}e^{i(\theta_1-\theta_2)}
\end{equation}

El complejo conjugado y la magnitud también son fáciles de calcular en este formato:

\begin{equation}
z^* = r e^{-i\theta}
\end{equation}

\begin{equation}
|z|^2 = z z^* = r e^{i\theta} r e^{-i\theta} = r^2 e^{i(\theta-\theta)} = r^2
\end{equation}

Aquí hay algunas otras fórmulas de números complejos útiles 

\begin{equation}
\cos\theta = \frac{e^{i\theta}+e^{-i\theta}}{2}
\end{equation}

\begin{equation}
\sin\theta = \frac{e^{i\theta}-e^{-i\theta}}{2i}
\end{equation}

\begin{equation}
r = \sqrt{\Re(z)^2+\Im(z)^2}
\end{equation}

\begin{equation}
r = \sqrt{\Re(z)^2+\Im(z)^2}
\end{equation}

\begin{equation}
\theta = 
     \begin{cases}
       \tan^{-1}\frac{\Im(z)}{\Re(z)} &\quad \Re(z)>0\\
        \tan^{-1}\frac{\Im(z)}{\Re(z)} +\frac{\pi}{2} &\quad \Re(z)<0 \ \text{and} \ \Im(z) \ge 0\\
        \tan^{-1}\frac{\Im(z)}{\Re(z)} - \frac{\pi}{2} &\quad \Re(z)<0 \ \text{and} \ \Im(z) < 0\\
     \end{cases}
\end{equation}

<a id="section-3"></a>
## Vectores y algebra vectorial

### Definición de Vectores e Interpretación Geométrica

Los vectores son objetos que tienen magnitud (o longitud) y dirección. Pueden existir en un espacio de 2, 3 o n dimensiones. 

<br> <img src="Figs/vecs.png" width="400" align="center" /> <br> 

Los vectores se representan como pares de números en dos dimensiones, triples de números en tres dimensiones y $n$-tupla de números en $n$ dimensiones.

<br> <img src="Figs/2n3vecs.png" width="400" align="center" /> <br> 

### Aritmética vectorial

#### Longitud o Magnitud de un Vector

La magnitud o longitud euclidiana de un vector en $n$ dimensiones viene dada por

\begin{equation}
\vert\vec{v}\vert = \sqrt{x_1^2 + x_2^2 + x_3^2 + \dots + x_n^2}
\end{equation}

Un vector unitario tiene una longitud de 1. Cualquier vector se puede convertir en un vector unitario dividiendo por su longitud

\begin{equation}
\hat{v} = \frac{\vec{v}}{\vert\vec{v}\vert}
\end{equation}

En la figura anterior de la derecha que ilustra el vector de 3 $\vec{v} =[x_1,x_2,x_3]$, la longitud de $\vec{v}$ viene dada por $|\vec{v}| = \sqrt{x_1^2 + x_2^2 + x_3^2}$. Su proyección sobre el plano $(x,y)$ también es un vector, con longitud $\sqrt{x_1^2 + x_2^2}$.

#### Multiplicación escalar de un vector

Para multiplicar un número por un vector simplemente multiplicamos cada componente del vector por el número. Si el valor absoluto del número es mayor que uno, la longitud aumenta. Si el valor absoluto del número es menor que uno, la longitud disminuye. Multiplicar por un número negativo apunta el vector en la dirección opuesta.

\begin{equation}
a\vec{v} =[ax_1,ax_2,ax_3,\dots,ax_n]
\end{equation}

<br> <img src="Figs/sclvecs.png" width="400" align="center" /> <br>

#### Suma y Resta de Vectores

Para sumar o restar a los vectores simplemente sume o reste sus componentes

\begin{equation}
\vec{v} =[v_1,v_2,v_3,\dots,v_n]
\end{equation}

\begin{equation}
\vec{w} =[w_1,w_2,w_3,\dots,w_n]
\end{equation}

\begin{equation}
\vec{v} \pm \vec{w} =[v_1\pm w_1,v_2\pm w_2,v_3\pm w_3,\dots,v_n\pm w_n]
\end{equation}

<br> <img src="Figs/addvecs.png" width="400" align="center" /> <br>

### Punto o producto interno y ángulo entre vectores

A menudo se desea conocer el ángulo entre dos vectores, por ejemplo para determinar si son perpendiculares o no. Esto se puede hacer usando el producto escalar de dos vectores definidos como

\begin{equation}
\vec{v} \cdot \vec{w} = \langle\vec{v},\vec{w}\rangle = v_1 w_1 + v_2 w_2 +  v_3 w_3 + \dots + v_n w_n = \sum_{i=1}^n v_i w_i
\end{equation}

Con esta definición, el ángulo entre los vectores viene dado por

\begin{equation}
\cos\theta = \frac{\vec{v}\cdot\vec{w}}{|\vec{v}| |\vec{w}|}
\end{equation}

La magnitud de un vector también se puede expresar convenientemente como un producto escalar

\begin{equation}
|\vec{v}| = \sqrt{\langle\vec{v},\vec{v}\rangle}
\end{equation}

Tenga en cuenta que si el producto escalar es igual a cero, entonces $\theta=\pi/2$ y los vectores son perpendiculares.

##### La ley de los cosenos es un producto punto

La ley de los cosenos se puede representar usando vectores como se muestra en la figura 

<br> <img src="Figs/losdp.png" width="200" align="left" /> <br>

\begin{equation}
|\vec{v} - \vec{w}|^2 = |\vec{v}|^2 + |\vec{w}|^2 - 2 |\vec{v}| |\vec{w}| \cos\theta
\end{equation}

\begin{equation}
(v_1-w_1)^2 + (v_2-w_2)^2 =  |\vec{v}| |\vec{w}| \cos\theta 
\end{equation}

Esto muestra que el producto escalar es otra forma de la Ley de los cosenos.


### Cross Products

The cross product of two vectors is a vector perpendicular to the two vectors. The cross product is only defined for vectors in three dimensions. It is calculated through the determinant

\begin{eqnarray}
\vec{v} \times \vec{w} & = & \begin{vmatrix}
\hat{i} & \hat{j} & \hat{k} \\
v_1 &  v_2 & v_3 \\
w_1 &  w_2 & w_3
\end{vmatrix} \\
& = & (v_2 w_3 - v_3 w_2) \hat{i} + (v_3 w_1 - v_1 w_3)\hat{j} + (v_1 w_2 - v_2 w_1)\hat{k}
\end{eqnarray}

In vector form, it can be written then

\begin{equation}
\vec{v} \times \vec{w} = \begin{bmatrix}
(v_2 w_3 - v_3 w_2) \\
(v_3 w_1 - v_1 w_3) \\
(v_1 w_2 - v_2 w_1)
\end{bmatrix}
\end{equation}


<br> <img src="Figs/crossprd.png" width="200" align="left" /> <br> A convenient way to remember the direction the resultant vector points from the cross product is to use the "right-hand rule".  Positioning the right hand fingers as shown in the figure, and assigning $\vec{v}$ and $\vec{w}$ to the index and middle fingers imagined to be in a plane, the resultant vector is in the direction the thumb points, perpendicular to the plane formed by the other fingers. 



<div class="alert alert-warning">
<font size="5"> <b> Test your understanding - Vectors <b> </font>
</div>

The look vector, $\vec{\ell}$,  to a target is given by

\begin{equation}
\vec{\ell} = \begin{bmatrix}
0 \\
3\\
-4 
\end{bmatrix} \nonumber
\end{equation}

then the unit vector, $\hat{\ell}$, pointing in this direction is

A.  $\hat{\ell} = \begin{bmatrix}
0 \\
\frac{3}{5}\\
\frac{4}{5}
\end{bmatrix}
$

B. $\hat{\ell} = \begin{bmatrix}
0 \\
\frac{3}{5}\\
-\frac{4}{5}
\end{bmatrix}
$

C. $\hat{\ell} = \begin{bmatrix}
0 \\
\frac{5}{3}\\
\frac{5}{4}
\end{bmatrix}
$
    
D.  $\hat{\ell} = \begin{bmatrix}
1 \\
1\\
1
\end{bmatrix}
$


In [7]:
 vec1

If $\vec{v}=( 0, \sin \theta_{\ell}, -\cos \theta_{\ell} )$, which of the following vectors is perpendicular to $\vec{v}$? 


A. $(0, \cos \theta_{\ell}, \sin \theta_{\ell} )$

B. $(0, -\cos \theta_{\ell}, \sin \theta_{\ell} )$

C. $ (\cos \theta_{\ell}, 0, \sin \theta_{\ell} )$ 

D. None of the above. 


In [8]:
vec2

<a id="section-4"></a>
## 4.0 Linear or Matrix Algebra 



### Definition of a Matrix

A matrix is simply an rectangular array of numbers.  The numbers may be real or complex.

\begin{equation}
A = \begin{bmatrix}
a_{11} & a_{12} & \dots & a_{1n} \\
{\color{blue} {a_{21}}} & {\color{blue} {a_{22}}} & \dots & {\color{blue} {a_{2n}}} \\
\vdots & \vdots & \vdots & \vdots \\
a_{m1} & a_{m2} & \dots & a_{mn} 
\end{bmatrix}
\end{equation}

The row index is the first index, running from 1 to $m$, as shown in blue, $a_{k,i},\  i=1,n$ for fixed $k$.  The column index is the second index, running from 1 to $n$; the column is defined as the numbers arranged vertically $a_{i,k},\  i=1,m$ for fixed $k$.

If $m=n$, the matrix is said to be square.  Mostly we will use $2 \times 2$ or $3 \times 3$ matrices.

Vectors are often represented as matricies with only 1 column, as we saw previously.

\begin{equation}
\vec{v} = \begin{bmatrix}
v_1 \\
v_2 \\
\vdots \\
v_n
\end{bmatrix}
\end{equation}

Matrices are useful because they can be used to represent linear transformations, for example rotations, or to solve systems of linear equations.

### Matrix Transpose

The transpose of an $m \times n$ matrix $A$, denoted $A^t$, is an $n \times m$ matrix obtained by interchanging rows and columns: the ijth entry in the transpose is given by

\begin{equation}
A^t_{ij} A_{ji}
\end{equation}

Written out in array form

\begin{equation}
\begin{bmatrix}
a_{11} & \dots & a_{1n} \\
\vdots & \vdots &  \vdots \\
a_{m1} & \dots & a_{mn} 
\end{bmatrix}^t
= \begin{bmatrix}
a_{11} &  \dots & a_{m1} \\
\vdots & \vdots & \vdots \\
a_{1n}  & \dots & a_{mn} 
\end{bmatrix}
\end{equation}

### Matrix Arithmetic

#### Addition and Subtraction

Two matrices of the same dimension can be added or subtracted simply by adding/subtracting the corresponding entries

\begin{equation}
A + B = \begin{bmatrix}
a_{11} & \dots & a_{1n} \\
\vdots & \vdots &  \vdots \\
a_{m1} & \dots & a_{mn} 
\end{bmatrix}
+ \begin{bmatrix}
b_{11} &  \dots & b_{1n} \\
\vdots & \vdots & \vdots \\
b_{m1}  & \dots & b_{mn} 
\end{bmatrix}
= \begin{bmatrix}
a_{11}+b_{11} &  \dots & a_{1n}+b_{1n} \\
\vdots & \vdots & \vdots \\
a_{m1}+b_{m1}  & \dots & a_{mn}+b_{mn} 
\end{bmatrix}
\end{equation}

A matrix can be multiplied by a number (real or complex) by multiplying all entries by that number

\begin{equation}
rA = r \begin{bmatrix}
a_{11} & \dots & a_{1n} \\
\vdots & \vdots &  \vdots \\
a_{m1} & \dots & a_{mn} 
\end{bmatrix}
=  \begin{bmatrix}
r a_{11} & \dots & r a_{1n} \\
\vdots & \vdots &  \vdots \\
r a_{m1} & \dots & r a_{mn} 
\end{bmatrix}
\end{equation}


#### Matrix Multiplication

Two matrices can be multiplied if the number of columns of the first matrix equals the number of rows of the second matrix

\begin{eqnarray}
C_{mn} = A_{mk} B_{kn} & = & 
\begin{bmatrix}
a_{11} & a_{12} & \dots & a_{1k} \\
a_{21} & a_{22} & \dots & a_{2k} \\
\vdots & \vdots &  \vdots & \vdots \\
{\color{blue} {a_{i1}}} & {\color{blue} {a_{i2}}} & {\color{blue} {\dots}} & {\color{blue} {a_{ik}}} \\
\vdots & \vdots &  \vdots & \vdots \\
a_{m1} & a_{m2} & \dots & a_{mk} 
\end{bmatrix}_{mk} \begin{bmatrix}
b_{11} & b_{12} & \dots & {\color{red} {b_{1j}}} & \dots & b_{1n} \\
b_{21} & b_{22} & \dots & {\color{red} {b_{2j}}} & \dots & b_{2n} \\
\vdots & \vdots &  \vdots & \vdots & \vdots & \vdots\\
b_{k1} & b_{k2} & \dots & {\color{red} {b_{kj}}} & \dots & b_{kn} \\ 
\end{bmatrix}_{kn} \\
& = & \begin{bmatrix}
c_{11} & \dots  & c_{1j} & \dots & b_{1n} \\
\vdots & \vdots & \vdots & \vdots & \vdots \\
c_{i1} & \dots  & \boxed{c_{ij}=\sum_{p=1}^k a_{ip} b_{pj}} & \dots & c_{2n} \\
\vdots & \vdots & \vdots & \vdots & \vdots \\
c_{m1} & \dots  & c_{mj} & \dots & c_{mn} \\ 
\end{bmatrix}_{mn}
\end{eqnarray}

The $i$th row (in blue) of $A$ and $j$th column (in red) of $B$ are multiplied as a dot product, shown in the box in $C$.  

Matrix multiplication is associative, but {\em not commutative}:
\begin{equation}
A(BC) = (AB)C
\end{equation}


### Matrix Inverse

A square matrix $A$ is said to be invertible if there exists a matrix $B$ such that
\begin{equation}
AB = BA = I
\end{equation}
where $I$ is the matrix with all ones on the diagonal and 0 elsewhere (note: $AI=A$)

If such a $B$ exists then it is called the inverse of $A$ and denoted $A^{-1}$. 
For a $2 \times 2$ matrix defined as
\begin{equation}
A=\begin{bmatrix}
a & b \\
c & d \\ 
\end{bmatrix}
\end{equation}
the determinant $\det(A) = ad-bc$, and the inverse is given by
\begin{equation}
A^{-1}=\frac{1}{ad-bc}\begin{bmatrix}
d & -b \\
-c & a \\ 
\end{bmatrix}
\end{equation}


In general if  $\det(A)\ne0$,  then an inverse exists. There are general formulas for computing the inverse, however they are not needed for this course.


### Rotation Matrices

Matrices can be used represent the rotation through an angle about an axis in either two or three dimensions. 

In two dimension the rotation matrices are of the form
\begin{equation}
R(\theta) = \begin{bmatrix}
\cos\theta & -\sin\theta \\
\sin\theta & \cos\theta \\ 
\end{bmatrix}
\end{equation}

Rotation matrices have two very special properties:
- The dot product of every row or column with itself equals 1.
- The inverse of a rotation matrix is equal to its transpose.
\begin{equation}
A^{-1} = A^t
\end{equation}

For the $2\times 2$ rotation matrix
\begin{equation}
R^{-1}(\theta) = R(-\theta) = \begin{bmatrix}
\cos\theta & \sin\theta \\
-\sin\theta & \cos\theta \\ 
\end{bmatrix}
\end{equation}


### Euler Angles and Three-dimensional Rotations

Euler angles provide a means of specifying an arbitrary 3-dimensional rotation matrix by specifying three angles and specified sequence of axes about which to rotate.

There are twelve possible right handed Euler angle sequences that may be used to specify any rotation matrix - and conversely every rotation matrix can be decomposed into any of the twelve possible Euler angle sequences.

Euler angle sequences are the sequence of axes about which to rotate, the twelve sequences are
<br> <img src="Figs/eulerangs.png" width="600" align="center" /> <br>



Rotations in three dimensions can be decomposed as the product of 3 elemental rotations (there are 12 right handed possibilities.) 

The elemental Rotation matrices are
\begin{equation}
R_x(\theta) = \begin{bmatrix}
1 & 0 & 0 \\
0 & \cos\theta & \sin\theta \\
0 & -\sin\theta & \cos\theta \\
\end{bmatrix} \qquad R_y(\theta) = \begin{bmatrix}
\cos\theta & 0 & -\sin\theta \\
0 & 1 & 0 \\
\sin\theta & 0 &  \cos\theta \\
\end{bmatrix}
\end{equation}

\begin{equation}
R_z(\theta) = \begin{bmatrix}
\cos\theta & \sin\theta & 0\\
-\sin\theta &  \cos\theta & 0\\
0 & 0 & 1 \\
\end{bmatrix}
\end{equation}


<a id="section-5"></a>
## 5.0 Basic Calculus 



### Derivatives

The derivative of a function provides a measure of how quickly a function is changing as well as to provide a way to approximate a function linearly in a neighborhood of a point: The derivative of a function gives the slope of the tangent line to the graph of the function at a point. 

The derivative is defined as 

\begin{equation}
\frac{d f}{dx}(x) = \dot{f}(x) = f^{\prime}(x) = \lim_{x \to 0} \frac{f(x+\Delta x) - f(x)}{\Delta x}
\end{equation}
<br> <img src="Figs/slope1.png" width="400" align="left" /> <br>  In region around $x_0$ a function can be approximated by its tangent line 
\begin{equation}
f(x) \approx f(x_o) + f^{\prime}(x_o)(x - x_o)
\end{equation}

### Product, Quotient and Chain Rules

The following three rules are very useful when computing derivatives 

Product Rule:

\begin{equation}
\frac{d}{dx} \left[ f(x) g(x) \right] = f(x) \frac{d g}{d x}(x) + g(x) \frac{d f}{d x}(x)
\end{equation}

Quotient Rule:

\begin{equation}
\frac{d}{dx} \left[ \frac{f(x)} {g(x)} \right] = \frac{g(x) \frac{d f}{d x}(x) - f(x) \frac{d g}{d x}(x)}{g^2(x)} 
\end{equation}

Chain Rule:

\begin{equation}
\frac{d}{dx}f(g(x)) = \frac{d f}{d x}(g(x)) \frac{d g}{d x}(x)
\end{equation}


### Derivatives of Some Useful Functions 

Polynomials

\begin{equation}
\frac{d}{dx} a = 0  \qquad \frac{d}{dx} x^n = nx^{n-1}
\end{equation}

Trigonometric Functions

\begin{equation}
\frac{d}{dx} \cos(x) = -\sin(x)  \qquad \frac{d}{dx} \sin(x) = \cos(x) \qquad \frac{d}{dx} \tan(x)  = \sec^2(x)
\end{equation}

Exponential and Log Functions

\begin{equation}
\frac{d}{dx} e^{x} = e^{x} \qquad \frac{d}{d x} \ln (x) = \frac{1}{x}
\end{equation}

### Partial Derivatives and Gradient

Sometimes we have functions that have more than one independent variable, *e.g.*, 

\begin{equation}
\rho(h_p,\theta_{\ell}) = \frac{h_p}{\cos \theta_{\ell}}
\end{equation}

The partial derivative of a functions is the derivative with respect to one of the variables while keeping the remining variables constant.

\begin{equation}
\frac{\partial f}{dx_i}(x_1,\cdots,x_n) = \lim_{x \to 0} \frac{f(x_1,\cdots,x_i+\Delta x,\cdots,x_n) - f(x_1,\cdots,x_n)}{\Delta x}
\end{equation}

The gradient is the vector of partial derivatives with respect to all the variables

\begin{equation}
\vec{\nabla} f(x_1,\cdots,x_n)= \vec{\nabla} f(\vec{x})= \left[ \frac{\partial f}{\partial x_1}(\vec{x}), \cdots, \frac{\partial f}{\partial x_n}(\vec{x}) \right]
\end{equation}

### Approximating a Multi-Variable Function

In defining the derivative above in one dimension as the local slope at a point, we saw that we could approximate the function there as:

\begin{equation}
f(x) \approx f(x_o) + f^{\prime}(x_o)(x - x_o)
\end{equation}

This can be extended to the multi-variable case, where the gradient can be used to approximate a function locally by the linear function

\begin{equation}
f(\vec{x}) \approx  f(\vec{x}_o) + \langle \vec{\nabla}f (\vec{x}_o) , \vec{x} - \vec{x}_o \rangle
\end{equation}

where the notation $\langle,\rangle$ represents the dot product, described in the linear algebra section. 

<br> <img src="Figs/mvslope.png" width="400" align="left" /> <br>This can be written in long form as:

\begin{equation}
f(x_1,\cdots,x_n)  \approx  f(x_{1_o},\cdots,x_{n_o}) + \sum \limits_{k=1}^{n} \frac{\partial f}{\partial x_k} (x_1,\cdots,x_n) (x_k- x_{k_o})
\end{equation}


### Integral Calculus

Integration is the inverse operation to differentiation, and can be used to find the area under the graph of a function. 

The anti-derivative, $F(x)$, of a function $f(x)$ is the function whose derivative is equal to $f(x)$

\begin{equation}
F(x) = \int  \!\! f(x) dx  \qquad \frac{d}{dx}F(x) = f(x)
\end{equation}


<br> <img src="Figs/defint.png" width="400" align="left" /> <br>The definite integral relates the signed area under a curve with the anti-derivative

\begin{equation}
\int \limits_{a}^{b} \!\!  f(x) dx = F(b) - F(a)
\end{equation}

### Integrals of Some Useful Functions 

Polynomials

\begin{equation}
\int \!\! x^n dx = \frac{1}{n+1} x^{n+1}
\end{equation}

Trigonometric Functions

\begin{equation}
\int \!\! \cos(x) dx = \sin (x) \qquad \int \!\! \sin(x) dx = -\cos (x)
\end{equation}

Exponential and Log Functions

\begin{equation}
\int \!\! e^{ax} dx = \frac{1}{a} e^{ax} \qquad \int \!\! \frac{1}{x} dx = \ln x
\end{equation}

<div class="alert alert-warning">
<font size="5"> <b> Test your understanding - Calculus <b> </font>
</div>

Consider the formula relating the interferometric phase, $\phi$, to the baseline length, $b$, look angle $\theta_{\ell}$ and baseline orientation angle $\alpha$ given by

\begin{equation}
\phi = \frac{4 \pi}{\lambda} b \sin(\theta_{\ell} - \alpha) \nonumber
\end{equation}

what is the partial derivative of the phase with respect to baseline orientation angle? 

A. $\frac{\partial \phi}{\partial \alpha} = -\frac{4 \pi}{\lambda} b \cos(\theta_{\ell} - \alpha)$

B. $\frac{\partial \phi}{\partial \alpha} =  \frac{4 \pi}{\lambda} b \sin(\theta_{\ell} - \alpha)$

C. $\frac{\partial \phi}{\partial \alpha} =  \frac{4 \pi}{\lambda} \sin(\theta_{\ell} - \alpha)$

D. $ \frac{\partial \phi}{\partial \alpha} = \frac{4 \pi}{\lambda} b \sin^2(\theta_{\ell} - \alpha)$


In [9]:
calc

<a id="section-6"></a>
## 6.0 Fourier Transforms 



In this module, we give a quick introduction to Fourier Transforms, the mathematical means of examining the spectrum of a time function.  We start with the basics of sine functions.

### Sinusoidal Functions

The sine function or sinusoid is a mathematical function that describes a smooth repetitive oscillation. It occurs often in pure mathematics, as well as physics, signal processing, electrical engineering and many other fields.
The most basic functional form is:
\begin{eqnarray}
s(t) & = & A \sin (\omega t + \phi)\\
     & = & A \sin (2\pi f t + \phi)
\end{eqnarray}

The following basic parameters describing the sinusoid 
- $A$, the amplitude, is the peak deviation of the function from 0.
- $\omega$, the angular frequency, specifies how many oscillations occur in a unit time interval, in radians per second
  - The angular frequency, $\omega$, period, $T$, and frequency, $f$, are related by
\begin{equation}
T=\frac{1}{f} = \frac{2\pi}{\omega}
\end{equation}
- $\phi$, the phase, specifies where in its cycle the oscillation begins at $t = 0$.

When $\phi\ne0$, the entire waveform appears to be shifted in time by the amount $\phi/\omega$ seconds. A negative value represents a delay, and a positive value represents an advance, "head-start".

In [10]:
def s(A,f,phi,t):
    return A*np.sin(2*np.pi()*f*t + phi)

#### Examples of sinusoids

Here are some examples of sinusoids and how they change according to changes in amplitude, phase, and frequency.

<br> <img src="Figs/sines.png" width="600" align="center" /> <br>

### Phasors

Euler’s Identity showed a relationship between sines, cosines and complex exponentials. It should not be a surprise then that sine functions can be views in terms of complex exponentials, which are called {\em phasors}. 

A phasor is a complex function of the form
\begin{equation}
z(t) = Ae^{i\omega t + \phi_0} = A\cos(\omega t + \phi_0) + i A \sin(\omega t + \phi_0)
\end{equation}
where $\phi_0$ is a constant phase offset.

The figure on the right shows the phasor representation in the complex plane.  As $t$ increases, for $\omega>0$ the phasor rotates counterclockwise. For $\omega<0$, it rotates clockwise.  In both cases, the phasor makes one rotation in T units of time.

<br> <img src="Figs/phasor.png" width="600" align="center" /> <br>

### Summing Sinusoids with Different Frequencies

By combining waves of different frequencies and amplitude we can generate interesting signals. The curves below show the function
\begin{equation}
f_{N}(t) = \frac{1}{2} + \sum_{k=1}^N\frac{2}{(2k-1)\pi}\sin((2k-1)t)
\end{equation}
for a number of values of $N$.  As higher frequency sinusoids are added, it is possible to represent functions with faster variations in time. 
<br> <img src="Figs/sinesum.png" width="600" align="center" /> <br>

### Frequency Content of a Signal

We just showed how we could generate interesting signals by combining sinusoids of different amplitude, frequency and phase. 

The inverse problem is the subject of Fourier analysis. That is: given a signal, what are the component frequencies present in the signal?
- A Fourier transform is sort of like a prism that separates white light into the colors of the rainbow. Each color is a different frequency or wavelength of light. 

In Synthetic Aperture Radar, Fourier Transforms play a key role in forming images.
Fourier Transforms are also important for understanding volumetric correlation in radar interferometry. 




### The Fourier Transform

The Fourier Transform is the mathematical tool that determines the frequency content of a time function. It transforms a function from the time domain to the frequency domain. The basic idea is very simple, and we use the analogy of a film reel to illustrate heuristically.  Imagine we create needle rotating in about an axis like a phasor rotating in the complex plain - for example a pressure gauge that rotates as pressure changes.  

<br> <img src="Figs/fourierframe.png" width="600" align="left" /> <br>If we were to film that rotating needle, phasors rotating at the same frequency as the file frame rate would be captured at the same orientation from frame to frame.  If we add up all the frames, this would be large signal.

Phasors rotating at a different frequency from the film frame rate would be captured at random moments, and would not add up to much if the frames were added.  For each angular frequency $\omega$ that is of interest, we change the frame rate to match that frequency and record the magnitude of the added frames.  The figure illustrates this idea. 

Summing frames is mathematically represented in the continuous domain by integration.  The definition of the Fourier Transform is given by

\begin{equation}
\mathcal{F}(s(t))(\omega) = S(\omega) \equiv \int_{-\infty}^{\infty} s(t) e^{-i\omega t} dt
\end{equation}

If $s(t) = e^{i\omega t}$, the integration would be over all time a constant value of 1; hence the integration is infinite.  For other values of $\omega$ it can be shown that the integral is 0. This is the mathematical expression of the heuristic example above.  For an arbitrary function, the Fourier Transform sorts the information into the  spectrum of frequency content in the function. 



### The sinc Function

The Fourier transform of a rectangular pulse leads to the classical {\em sinc} function and shows the relationship between resolution and bandwidth.  The time domain function is 

<br> <img src="Figs/tdsinc.png" width="300" align="left" /> <br>\begin{eqnarray}
\mathcal{F}({\text{rect}}(t))(\omega) & = & \int_{-\infty}^{\infty} {\text{rect}}(t) e^{-i\omega t} dt\\
& = & \int_{-\tau/2}^{\tau/2} e^{-i\omega t} dt\\
& = & -\frac{1}{i\omega} e^{-i\omega t}\Big\rvert_{-\tau/2}^{\tau/2}\\
& = & -\frac{1}{i\omega} (e^{-i\omega \tau/2} - e^{i\omega \tau/2})\\
& = & \frac{2}{\omega} \frac{(e^{-i\omega \tau/2} - e^{i\omega \tau/2})}{2i}\\
& = & \frac{2}{\omega} \sin \bigl(\frac{\omega \tau}{2}\bigr) \\
& = & \tau \frac{\sin \bigl(\frac{\omega \tau}{2}\bigr)}{\frac{\omega \tau}{2}} = \tau\  {\text{sinc}} \bigl(\frac{\omega \tau}{2}\bigr)
\end{eqnarray}

<br> <img src="Figs/fdsinc.png" width="300" align="left" /> <br>Note that as the pulse becomes narrower, i.e., as $\tau$ becomes smaller and there is finer time resolution, the spectrum becomes broader.  This illustrates a general principle that to have finer spatial resolution requires a broader band signal.

### Useful Fourier Transform Relationships

Here are a few Fourier transform properties that are useful. 

Inverse Transform: $\qquad\mathcal{F}^{-1}(S(\omega))(t) =  \int_{-\infty}^{\infty} S(\omega) e^{i\omega t} d\omega$

Linearity: $\qquad \mathcal{F}(as(t) + bw(t)) = a \mathcal{F}(s(t)) + b \mathcal{F}(w(t))$

Shift Property: $\qquad \mathcal{F}(s(t\pm t_0))  = S(\omega) e^{i\pm\omega t_0}$

Time Reversal:  $\qquad \mathcal{F}(s(-t)) = S(-\omega)$

Scaling: $\qquad \mathcal{F}(s(at)) = \frac{1}{a}S\bigl(\frac{\omega}{a}\bigr)$

Conjugation:  $\qquad \mathcal{F}(s^*(t)) = S^*(-\omega)$

Parseval's Theorem of energy conservation: $\qquad \int_{-\infty}^{\infty} |s(t)|^2 dt = \frac{1}{2\pi} \int_{-\infty}^{\infty} |S(\omega)|^2 d\omega$

<a id="section-7"></a>
## 7.0 Decibels

### What is a dB?

In engineering and physics, we often encounter quantities that can vary by many orders of magnitude. 

It turns out to be convenient to express such quantities in terms of decibels or dBs.

A decibel is a logarithmic unit used to describe a ratio. For example, power is often measured in dB relative to a reference of 1 Watt.

\begin{equation}
P_{\text{dB}} = 10 \log_{10} P
\end{equation}

Since logarithms convert multiplications into additions, doubling a quantity increases it by 3 dB

\begin{equation}
10 \log_{10} 2P = 10 \log_{10} P + 10 \log_{10} 2 \approx 10 \log_{10} P + 3{\text{dB}}
\end{equation}

Sometimes a 20 is used instead of a 10 in converting to dB. This is because the quantity needs to be squared before converting.

\begin{equation}
P_{\text{dB}} = 10 \log_{10} V^2 = 20 \log_{10} V
\end{equation}

This is usually encountered when considering power ratios based on signal amplitudes or voltages.

<div class="alert alert-warning">
<font size="5"> <b> Test your understanding - deciBels <b> </font>
</div>

An amplifier increases the power of a returned radar echo by a factor of 15. By how many dB has the power increased? 

A. $20 \log_{10}(15) $ dB

B. 3 dB

C. 15 dB

D. $10 \log_{10}(15) $ dB
  

In [11]:
dB